# Road Following - Live demo (Jetson-category TensorRT) with Jetbot anti-collision trained TRT model

Jetracer cagtegory scritpt to operate jetbot interference  and chose category (e.g. follow street or object) to run the Jetbot with Anti-collision trained Jetbot model

Load the TRT optimized model by executing the cell below - # TensorRT
## NEED TO CHOSE CATEGORY AND PRE TRAINED MODEL

In [ ]:
import torch
from torch2trt import TRTModule

device = torch.device('cuda')

#CATEGORIES = ['apex','bottle']

CATEGORIES = ['road','bottle']

# if no Categories then activate like Jetbot trained model then:
#CATEGORIES = []
#model_trt.load_state_dict(torch.load('best_steering_model_xy_trt.pth')) #jetbot model from training and build TRT without category

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_following_model_trt.pth')) #jetracer for jetbot model from training and build with TRT

model_trt_collision = TRTModule()
model_trt_collision.load_state_dict(torch.load('best_model_trt.pth')) # anti collision model traubed for one object to block and street signals (ground, strips) free


### Creating the Pre-Processing Function

We have now loaded our model, but there's a slight issue. The format that we trained our model doesnt exactly match the format of the camera. To do that, we need to do some preprocessing. This involves the following steps:

1. Convert from HWC layout to CHW layout
2. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
3. Transfer the data from CPU memory to GPU memory
4. Add a batch dimension

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [ ]:
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()

In [ ]:
import ipywidgets
import ipywidgets.widgets as widgets
import time
import IPython
from IPython.display import display
import traitlets

target_widget = widgets.Image(format='jpeg', width=224, height=224)
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='horizontal', description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * -224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

#time.sleep(1)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)


display(widgets.HBox([target_widget]))
d2 = IPython.display.display("", display_id=2)

We'll also create our robot instance which we'll need to drive the motors.

In [ ]:
from jetbot import Robot

robot = Robot()

In [ ]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.09, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.24, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

blocked_slider = ipywidgets.FloatSlider(description='blocked', min=0.0, max=120.0, orientation='horizontal')
stopduration_slider= ipywidgets.IntSlider(min=1, max=1000, step=1, value=200 description='Manu. time stop') #anti-collision stop time
block_threshold= ipywidgets.FloatSlider(min=0, max=2, step=0.1, value=0.9, description='Manu. bl threshold') #anti-collision block probability

display(ipywidgets.HBox([ blocked_slider, stopduration_slider, block_threshold]))

#create new New View for Output


Next, let's display some sliders that will let us see what JetBot is thinking.  The x and y sliders will display the predicted x, y values.

The steering slider will display our estimated steering value.  Please remember, this value isn't the actual angle of the target, but simply a value that is
nearly proportional.  When the actual angle is ``0``, this will be zero, and it will increase / decrease with the actual angle.  

In [ ]:
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='horizontal', description='speed')
category_widget = ipywidgets.Dropdown(options=np.array(CATEGORIES), description='category')


display(y_slider, x_slider, speed_slider, steering_slider)

#choose category for road or object following
category_execution_widget = ipywidgets.VBox([category_widget])

display(category_execution_widget)

In [ ]:
def start_category(change):
    global category_index
    if not CATEGORIES: 
        category_index=0
    else:
        category_index=CATEGORIES.index(category_widget.value)
    return

category_widget.observe(start_category, names='value')

#repeated for initialization
if not CATEGORIES: 
    print("List is empty.")
    category_index=0;
else:
    category_index=CATEGORIES.index(category_widget.value)
  

Next, we'll create a function that will get called whenever the camera's value changes. This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. Compute the approximate steering value
4. Control the motors using proportional / derivative control (PD)

In [ ]:
import time
import torch
import torch.nn as nn
from torchvision import models
from torch.hub import load_state_dict_from_url
from PIL import Image
import cv2
import numpy as np
from matplotlib import pyplot as plt


count=0
count_stops=0
stop_time=50 #(for how many frames the bot should go backwards, see and of script)
angle = 0.0
angle_last = 0.0
go_on=1
max_x=camera.width
min_x=0
max_y=camera.height
min_y=0
sum_bottle=0.0
x=0
y=0

def execute(change):
    global angle, angle_last, category_index, count, count_stops,stop_time,go_on,block_threshold,stop_time,max_x, min_x,max_y,min_y,x,y
    count +=1
    t1 = time.time()
    
    image = change['new']
    image_preproc = preprocess(image)
    #anti_collision model)-----
    y_collison=model_trt_collision(image_preproc)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y_collison = F.softmax(y_collison, dim=1)    
    prob_blocked = float(y_collison.flatten()[0])
    blocked_slider.value = prob_blocked
    stop_time=stopduration_slider.value
   
    if go_on==1:
        if prob_blocked > block_threshold.value: #in case collision avoidance model hast recognized an object (not specific one) (prob_blocked) then stop for some time about ~ frames*0.05ms 
            count_stops +=1
            x=0.0 #set steering zero
            y=0.0 #set steering zero
            speed_slider.value=0.0 # set speed zero or negative or turn
            go_on=2
            #anti_collision-------
        else: #start road following if no object
            go_on=1
            count_stops=0
            #-------------------
            xy = model_trt(image_preproc).detach().float().cpu().numpy().flatten()  
            x = float(xy[2 * category_index])
            y = float(xy[2 * category_index + 1] )  
            x = int(max_x * (x / 2.0 + 0.5))
            y = int(max_y * (y / 2.0 + 0.5))
            speed_slider.value = speed_gain_slider.value #
    else:
        count_stops=count_stops+1
        if count_stops<stop_time: #how many frames bot should pause
            x=0.0 #set steering zero
            y=0 #set steering zero
            speed_slider.value=0 # set speed zero or negative or turn
        else:
            go_on=1
            count_stops=0
        
    
    x_joysticklike=((x-max_x/2.0)-min_x)/(max_x-min_x)
    y_joysticklike=((max_y-y)-min_y)/(max_y-min_y)     
    x_slider.value = x_joysticklike
    y_slider.value = y_joysticklike
    
    #---------        
        
    angle = np.arctan2(x_joysticklike, y_joysticklike)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle 
    steering_slider.value = pid + steering_bias_slider.value
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
    #---------    
    
    t2 = time.time()
    s = f"""{int(1/(t2-t1))} FPS"""
    d2.update(IPython.display.HTML(s) )
    
    
execute({'new': camera.value})

The jupyter plot shows the x and y values, x value is more variable, y value is more or less the same (y might be useful later for velocity regulation),
The initialization of jupyterplot takes >30 sec, so need to wait, once 0 FPSS appears, activate system with camera.observe below


Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

>WARNING: This code will move the robot!! Please make sure your robot has clearance and it is on Lego or Track you have collected data on. The road follower should work, but the neural network is only as good as the data it's trained on!

In [ ]:
camera.observe(execute, names='value')

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame. 

You can now place JetBot on  Lego or Track you have collected data on and see whether it can follow track.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [ ]:
import time
camera.unobserve(execute, names='value')
time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
robot.stop()

### Conclusion
That's it for this live demo! Hopefully you had some fun seeing your JetBot moving smoothly on track follwing the road!!!

If your JetBot wasn't following road very well, try to spot where it fails. The beauty is that we can collect more data for these failure scenarios and the JetBot should get even better :)